# Internal Dependencies
<br>  

### References
- [Analyze java package metrics in a graph database](https://joht.github.io/johtizen/data/2023/04/21/java-package-metrics-analysis.html)
- [Calculate metrics](https://101.jqassistant.org/calculate-metrics/index.html)
- [py2neo](https://py2neo.org/2021.1/)

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plot
from neo4j import GraphDatabase

In [2]:
# Please set the environment variable "NEO4J_INITIAL_PASSWORD" in your shell 
# before starting jupyter notebook to provide the password for the user "neo4j". 
# It is not recommended to hardcode the password into jupyter notebook for security reasons.

driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", os.environ.get("NEO4J_INITIAL_PASSWORD")))
driver.verify_connectivity()

In [3]:
def get_cypher_query_from_file(cypherFileName):
    with open(cypherFileName) as file:
        return ' '.join(file.readlines())

In [4]:
def query_cypher_to_data_frame(filename):
    records, summary, keys = driver.execute_query(get_cypher_query_from_file(filename))
    return pd.DataFrame([r.values() for r in records], columns=keys)

In [5]:
#The following cell uses the build-in %html "magic" to override the CSS style for tables to a much smaller size.
#This is especially needed for PDF export of tables with multiple columns.

In [6]:
%%html
<style>
/* CSS style for smaller dataframe tables. */
.dataframe th {
    font-size: 8px;
}
.dataframe td {
    font-size: 8px;
}
</style>

## Artifacts

### Table 1

- List all the artifacts this notebook is based on

In [7]:
query_cypher_to_data_frame("../cypher/List_all_existing_artifacts.cypher")

,artifactName,packages,types
0,axon-configuration-4.8.0.jar,1,39
1,axon-test-4.8.0.jar,8,87
2,axon-eventsourcing-4.8.0.jar,9,130
3,axon-disruptor-4.8.0.jar,1,22
4,axon-modelling-4.8.0.jar,10,150
5,axon-messaging-4.8.0.jar,64,762


## Cyclic Dependencies

Cyclic dependencies occur when one package uses a class of another package and vice versa. 
These dependencies can lead to a lot of trouble when one of these packages needs to be changed.

### Table 2
- List packages with cyclic dependencies as an overview

In [8]:
query_cypher_to_data_frame("../cypher/Cyclic_Dependencies/Cyclic_Dependencies_as_List.cypher")

,packageName,dependentPackageName,forwardToBackwardBalance,numberForward,numberBackward,forwardDependencies,backwardDependencies
0,org.axonframework.queryhandling,org.axonframework.messaging.responsetypes,0.882353,16,1,"[SubscriptionQueryMessage->ResponseType, Gener...",[ConvertingResponseMessage->QueryResponseMessage]
1,org.axonframework.eventhandling,org.axonframework.tracing,0.857143,13,1,"[SimpleEventBus$Builder->SpanFactory, Abstract...",[NestingSpanFactory->EventMessage]
2,org.axonframework.eventhandling,org.axonframework.messaging,0.853659,38,3,[StreamingEventProcessor->StreamableMessageSou...,"[Headers->EventMessage, Headers->DomainEventMe..."
3,org.axonframework.eventhandling,org.axonframework.messaging.annotation,0.840000,23,2,"[ResetHandler->MessageHandler, SimpleEventHand...",[SourceIdParameterResolverFactory$SourceIdPara...
4,org.axonframework.queryhandling,org.axonframework.tracing,0.800000,9,1,"[SimpleQueryUpdateEmitter->Span, SimpleQueryUp...",[SpanUtils->QueryMessage]
5,org.axonframework.eventsourcing,org.axonframework.eventsourcing.eventstore,0.777778,16,2,"[AggregateSnapshotter$Builder->EventStore, Agg...","[DomainEventStream->EventStreamUtils, Abstract..."
6,org.axonframework.deadline,org.axonframework.tracing,0.750000,7,1,"[SimpleDeadlineManager$DeadlineTask->Span, Sim...",[SpanUtils->DeadlineMessage]
7,org.axonframework.commandhandling.callbacks,org.axonframework.commandhandling,0.733333,13,2,"[FutureCallback->CommandResultMessage, FutureC...","[SimpleCommandBus$Builder->LoggingCallback, Si..."
8,org.axonframework.commandhandling,org.axonframework.tracing,0.666667,5,1,"[SimpleCommandBus->Span, SimpleCommandBus->Spa...",[SpanUtils->CommandMessage]
9,org.axonframework.eventhandling,org.axonframework.serialization,0.647059,14,3,[AbstractSequencedDomainEventEntry->Serializer...,[GapAwareTrackingTokenConverter->GapAwareTrack...


### Table 3
- List packages with cyclic dependencies with every dependency in a separate row sorted by the easiest and most valuable resolvable dependency first

In [9]:
query_cypher_to_data_frame("../cypher/Cyclic_Dependencies/Cyclic_Dependencies_as_unwinded_List.cypher").head(60)

,packageName,dependentPackageName,dependency,forwardToBackwardBalance,numberForward,numberBackward
0,org.axonframework.queryhandling,org.axonframework.messaging.responsetypes,QueryResponseMessage<-ConvertingResponseMessage,0.882353,16,1
1,org.axonframework.queryhandling,org.axonframework.messaging.responsetypes,SubscriptionQueryMessage->ResponseType,0.882353,16,1
2,org.axonframework.queryhandling,org.axonframework.messaging.responsetypes,GenericStreamingQueryMessage->PublisherRespons...,0.882353,16,1
3,org.axonframework.queryhandling,org.axonframework.messaging.responsetypes,GenericStreamingQueryMessage->ResponseType,0.882353,16,1
4,org.axonframework.queryhandling,org.axonframework.messaging.responsetypes,SimpleQueryUpdateEmitter->OptionalResponseType,0.882353,16,1
5,org.axonframework.queryhandling,org.axonframework.messaging.responsetypes,SimpleQueryUpdateEmitter->MultipleInstancesRes...,0.882353,16,1
6,org.axonframework.queryhandling,org.axonframework.messaging.responsetypes,SimpleQueryUpdateEmitter->PublisherResponseType,0.882353,16,1
7,org.axonframework.queryhandling,org.axonframework.messaging.responsetypes,SimpleQueryUpdateEmitter->ResponseType,0.882353,16,1
8,org.axonframework.queryhandling,org.axonframework.messaging.responsetypes,QueryGateway->ResponseTypes,0.882353,16,1
9,org.axonframework.queryhandling,org.axonframework.messaging.responsetypes,QueryGateway->ResponseType,0.882353,16,1


## Interface Segregation Candidates

Well known from [Design Principles and Design Patterns by Robert C. Martin](http://staff.cs.utu.fi/~jounsmed/doos_06/material/DesignPrinciplesAndPatterns.pdf), the *Interface Segregation Principle* suggests that software components should have narrow, focused interfaces rather than large, general-purpose ones. The goal is to minimize the dependencies between components and increase modularity, flexibility, and maintainability.

Smaller, focused and purpose-driven interfaces

- make it easier to modify individual components without affecting the rest of the system.
- make it clearer which client is affected by which change.
- don’t force their clients to depend on methods they don’t need.
- reduce the scope of changes since a change to one component doesn’t affect others.
- lead to a more loosely coupled architecture that is easier to understand and maintain.

Reference: [Analyze java package metrics in a graph database](https://joht.github.io/johtizen/data/2023/04/21/java-package-metrics-analysis.html#interface-segregation)

### How to apply the results

If just one method of a type is used, especially in many places, then the result of this method can be used to call e.g. a method or constuct an object instead of using the whole object and then just calling that single method.

If there are a couple of methods that are used for a distinct purpose, those could be factored out into a separate interface. The original type can extended/implement the new interface so that there are no breaking changes. Then all the callers, that use only this group of methods, can be changed to the new interface.

### Table 4
- List top 20 most used combinations of methods of larger Types that might benefit from *Interface Segregation*

In [10]:
query_cypher_to_data_frame("../cypher/Candidates_for_Interface_Segregation.cypher").head(20)

,fullDependentTypeName,declaredMethods,calledMethodNames,calledMethods,callerTypes
0,org.axonframework.commandhandling.CommandMessage,9,[getCommandName],1,18
1,org.axonframework.eventhandling.EventMessage,9,"[getTimestamp, getIdentifier]",2,10
2,org.axonframework.eventhandling.DomainEventMes...,10,[getSequenceNumber],1,9
3,org.axonframework.eventhandling.TrackedEventMe...,10,[trackingToken],1,8
4,org.axonframework.eventhandling.EventMessage,9,[getIdentifier],1,8
5,org.axonframework.eventhandling.DomainEventMes...,10,"[getType, getSequenceNumber, getAggregateIdent...",3,6
6,org.axonframework.messaging.ResultMessage,9,"[isExceptional, exceptionResult]",2,6
7,org.axonframework.commandhandling.GenericComma...,14,[asCommandResultMessage],1,5
8,org.axonframework.deadline.GenericDeadlineMessage,11,[asDeadlineMessage],1,5
9,org.axonframework.eventhandling.TrackedEventMe...,12,[trackingToken],1,4


## Package Usage

### Types that are used by multiple packages

#### Table 5
- List the top 20 packages that are used by the highest count of different packages 

In [11]:
query_cypher_to_data_frame("../cypher/Package_Usage/List_types_that_are_used_by_many_different_packages.cypher").head(20)

,dependentType.fqn,dependentType.name,dependentTypeLabels,numberOfUsingPackages
0,org.axonframework.common.BuilderUtils,BuilderUtils,"[Type, File, Java, ByteCode, Class]",44
1,org.axonframework.messaging.Message,Message,"[Type, File, Java, GenericDeclaration, ByteCod...",39
2,org.axonframework.common.AxonConfigurationExce...,AxonConfigurationException,"[Type, File, Java, ByteCode, Class]",37
3,org.axonframework.messaging.MetaData,MetaData,"[Type, File, Java, ByteCode, Class]",33
4,org.axonframework.eventhandling.EventMessage,EventMessage,"[Type, File, Java, GenericDeclaration, ByteCod...",32
5,org.axonframework.messaging.unitofwork.UnitOfWork,UnitOfWork,"[Type, File, Java, GenericDeclaration, ByteCod...",31
6,org.axonframework.serialization.Serializer,Serializer,"[Type, File, Java, ByteCode, Interface]",29
7,org.axonframework.common.Assert,Assert,"[Type, File, Java, ByteCode, Class]",27
8,org.axonframework.common.transaction.Transacti...,TransactionManager,"[Type, File, Java, ByteCode, Interface]",27
9,org.axonframework.serialization.SerializedObject,SerializedObject,"[Type, File, Java, GenericDeclaration, ByteCod...",25


### Packages that are used by multiple artifacts

#### Table 6
- List the top 20 artifacts that only use a few (compared to all existing) packages of another artifact

In [12]:
query_cypher_to_data_frame("../cypher/Package_Usage/How_many_packages_compared_to_all_existing_are_used_by_dependent_artifacts.cypher").head(20)

,artifactName,dependentArtifactName,dependentPackages,dependentArtifactPackages,packageUsagePercentage,dependentFullQualifiedPackageNames,dependentPackageNames
0,axon-disruptor-4.8.0,axon-messaging-4.8.0,9,64,0.140625,"[org.axonframework.commandhandling, org.axonfr...","[commandhandling, transaction, caching, unitof..."
1,axon-test-4.8.0,axon-messaging-4.8.0,10,64,0.156250,"[org.axonframework.commandhandling, org.axonfr...","[commandhandling, messaging, eventhandling, co..."
2,axon-eventsourcing-4.8.0,axon-modelling-4.8.0,2,10,0.200000,"[org.axonframework.modelling.command, org.axon...","[command, inspection]"
3,axon-disruptor-4.8.0,axon-modelling-4.8.0,2,10,0.200000,"[org.axonframework.modelling.command, org.axon...","[command, inspection]"
4,axon-test-4.8.0,axon-eventsourcing-4.8.0,2,9,0.222222,"[org.axonframework.eventsourcing, org.axonfram...","[eventsourcing, eventstore]"
5,axon-disruptor-4.8.0,axon-eventsourcing-4.8.0,2,9,0.222222,"[org.axonframework.eventsourcing.eventstore, o...","[eventstore, eventsourcing]"
6,axon-modelling-4.8.0,axon-messaging-4.8.0,18,64,0.281250,"[org.axonframework.common.annotation, org.axon...","[annotation, tracing, property, messaging, com..."
7,axon-eventsourcing-4.8.0,axon-messaging-4.8.0,20,64,0.312500,"[org.axonframework.common, org.axonframework.e...","[common, eventhandling, event, serialization, ..."
8,axon-configuration-4.8.0,axon-eventsourcing-4.8.0,4,9,0.444444,"[org.axonframework.eventsourcing, org.axonfram...","[eventsourcing, jpa, snapshotting, eventstore]"
9,axon-test-4.8.0,axon-modelling-4.8.0,5,10,0.500000,"[org.axonframework.modelling.command, org.axon...","[command, inspection, saga, repository, inmemory]"


### Packages that are used by multiple artifacts

#### Table 7
- List the top 20 packages that only use a few (compared to all existing) types of another package 

In [13]:
query_cypher_to_data_frame("../cypher/Package_Usage/How_many_classes_compared_to_all_existing_in_the_same_package_are_used_by_dependent_packages_across_different_artifacts.cypher").head(20)

,artifactName,dependentArtifactName,packageName,dependentPackage.fqn,dependentTypes,dependentPackageTypes,typeUsagePercentage,dependentTypeNames
0,axon-eventsourcing-4.8.0,axon-messaging-4.8.0,org.axonframework.eventsourcing.snapshotting,org.axonframework.eventhandling,1,93,0.010753,[org.axonframework.eventhandling.DomainEventData]
1,axon-modelling-4.8.0,axon-messaging-4.8.0,org.axonframework.modelling.saga.metamodel,org.axonframework.eventhandling,1,93,0.010753,[org.axonframework.eventhandling.EventMessage]
2,axon-test-4.8.0,axon-messaging-4.8.0,org.axonframework.test.matchers,org.axonframework.eventhandling,1,93,0.010753,[org.axonframework.eventhandling.EventMessage]
3,axon-eventsourcing-4.8.0,axon-modelling-4.8.0,org.axonframework.eventsourcing.conflictresolu...,org.axonframework.modelling.command,1,51,0.019608,[org.axonframework.modelling.command.Conflicti...
4,axon-eventsourcing-4.8.0,axon-modelling-4.8.0,org.axonframework.eventsourcing.eventstore.jdbc,org.axonframework.modelling.command,1,51,0.019608,[org.axonframework.modelling.command.Concurren...
5,axon-modelling-4.8.0,axon-messaging-4.8.0,org.axonframework.modelling.command.legacyjpa,org.axonframework.eventhandling,2,93,0.021505,[org.axonframework.eventhandling.DomainEventSe...
6,axon-modelling-4.8.0,axon-messaging-4.8.0,org.axonframework.modelling.command,org.axonframework.eventhandling,2,93,0.021505,[org.axonframework.eventhandling.DomainEventSe...
7,axon-test-4.8.0,axon-messaging-4.8.0,org.axonframework.test.eventscheduler,org.axonframework.eventhandling,2,93,0.021505,[org.axonframework.eventhandling.GenericEventM...
8,axon-eventsourcing-4.8.0,axon-messaging-4.8.0,org.axonframework.eventsourcing.conflictresolu...,org.axonframework.eventhandling,2,93,0.021505,"[org.axonframework.eventhandling.EventMessage,..."
9,axon-eventsourcing-4.8.0,axon-messaging-4.8.0,org.axonframework.eventsourcing.eventstore,org.axonframework.messaging,1,35,0.028571,[org.axonframework.messaging.StreamableMessage...
